In [ ]:
from astropy import constants as const
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import fits

## Exercise 1
Let's practice some more plotting skills, now incorporating units.

A. Write a function that takes an array of frequencies and spits out the Planck distribution. That's this equation:

$ B(\nu, T) = \frac{2h\nu^3/c^2}{e^{\frac{h\nu}{k_B T}} - 1}$
 
 
This requires you to use the Planck constant, the Boltzmann constant, and the speed of light from astropy. Make sure they are all in cgs.

B. Plot your function in log-log space for T = 25, 50, and 300 K. The most sensible frequency range is about $10^5$ to $10^{15}$ Hz. Hint: if your units are correct, your peak values of B(T) should be on the order of $10^{-10}$. Make sure everything is labelled.

In [ ]:
from astropy import constants as const 

h = const.h.cgs.value
kB = const.k_B.cgs.value
c = const.c.cgs.value

def planck(nu, T):
    num = 2 * h * nu**3 / c**2
    denom = np.exp( (h * nu) / (kB * T)) - 1
    return num / denom

nu_array = np.logspace(5, 15, 1000)
T1 = 25.
T2 = 50.
T3  = 300.

B1 = planck(nu_array, T1)
B2 = planck(nu_array, T2)
B3 = planck(nu_array, T3)

# plot it for 3 dust temperatures

plt.loglog(nu_array, B1, 'k:', label='T=25 K')
plt.loglog(nu_array, B2, '--', color='orchid', label='T = 50 K')
plt.loglog(nu_array, B3, '-', color='lightseagreen', label='T = 300K')

plt.legend(fontsize=14)

plt.ylim(1e-36, 1e-7)

plt.xlabel(r'$\nu$ (Hz)', fontsize=16)
plt.ylabel(r'B($\nu$, T)', fontsize=16)

# Exercise 2

Let's put everything together now! Here's a link to the full documentation for FITSFigure, which will tell you all of the customizable options: http://aplpy.readthedocs.io/en/stable/api/aplpy.FITSFigure.html. Let's create a nice plot of M51 with a background optical image and X-ray contours overplotted.

The data came from here if you're interested: http://chandra.harvard.edu/photo/openFITS/multiwavelength_data.html

A. Using astropy, open the X-RAY data (m51_xray.fits). Flatten the data array and find its standard deviation, and call it sigma.

B. Using aplpy, plot a colorscale image of the OPTICAL data. Choose a colormap that is visually appealing (list of them here: https://matplotlib.org/2.0.2/examples/color/colormaps_reference.html). Show the colorbar. 

C. Plot the X-ray data as contours above the optical image. Make the contours spring green with 80% opacity and dotted lines. Make the levels go from 2$\sigma$ to 10$\sigma$ in steps of 2$\sigma$. (It might be easier to define the levels array before show_contours, and set levels=levels.)

In [ ]:
# A.
from astropy.io import fits

xray_data = fits.open('m51_xray.fits')[0].data
sigma = np.std(xray_data.flatten())

# B.
import aplpy

xray = aplpy.FITSFigure('../m51_optical_B.fits')
xray.show_colorscale(vmid=0, cmap='plasma')

# C.
levels = np.arange(2, 12, 2) * sigma
xray.show_contour('../m51_xray.fits', levels=levels, colors='springgreen', linewith=0.25, alpha=0.8)

## Exercise 3: Spectral Analysis

Code and Exercise provided to us by Olivia Cooper

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
%matplotlib inline
import pandas as pd
import astropy
from astropy.io import fits
from astropy.visualization import ZScaleInterval
from astropy.stats import sigma_clip
from scipy.optimize import curve_fit
#plt.style.use('cooper-paper.mplstyle')

# Plotting spectra in 1D and 2D

In this example, we'll be looking at a source from a MOSFIRE program called The MOSFIRE Deep Evolution Field Survey (MOSDEF). This data set contains hundreds of galaxies (find the data here: https://mosdef.astro.berkeley.edu/for-scientists/data-releases/), but we'll look at just one.

In [ ]:
# load in the spectrum in 1D and 2D for object 2335 in the MOSDEF catalog

#This is the one that has 2335.2d.fits extension
hdu2 = fits.open('data/co2_01.H.2335.2d.fits') # 2d spectrum

#This is the one that has 2335.2d.fits extension
hdu1 = fits.open('data/co2_01.H.2335.ell.1d.fits') # 1d spectrum

Let's first look at the 2D spectrum. Based on the header, we want to look at the science frame, extension 1. (If you want to see the noise frames etc, type in the appropriate index for the extension noted in the header!) I wrote a quick function to help us plot it.

In [ ]:
# it's a good idea to check out the header to see how the data are set up, 
#Open up the header for the 2D file

#Code here

hdu2[0].header

In [ ]:
def show_2d(image, lower=-1, upper=2):
    """
    Plot grayscaled 2D spectrum

    Parameters
    ----------
    image : fits data
        Fits data for 2D image.
        
    lower : float
        Lower value to dictate vmin for grayscale. The default is -1.
    
    upper : float
        Upper value to dictate vmax for grayscale. The default is 2.

    Returns
    -------
    Plot of 2D spectrum.

    """

    sample = sigma_clip(image) 
    vmin = sample.mean() + lower * sample.std()
    vmax = sample.mean() + upper * sample.std()
    plt.figure(figsize=(15, 3))
    plt.imshow(image, origin='lower', cmap='gray', aspect='auto', vmin=vmin, vmax=vmax)
    plt.xlabel('Column Number')
    plt.ylabel('Row Number')
    plt.grid(False)

In [ ]:
# display the 2D spectrum

show_2d(hdu2[1].data) 

You'll notice a few things: 
1) Going across the image there is a white line -- this is the continuum! That is the signal or light coming from the galaxy, all spread out in wavelength space.

2) Above and below the continuum there are black lines going across. This is the negative of the signal, and is a feature we expect to see in this type of data if the signal is real (it's a product of the ABBA set-up...ask us if you want to know more!).

3) There are TV static noisy vertical features -- these are the residuals of sky lines, of which there are a ton in the near-infrared! These sky lines have to be removed (as they are here) so we can extract the underlying signal from the galaxy.

4) Along the continuum, there are a couple bright spots. Finding and identifying these features is our goal! These are emission lines, and based on the exact positions of these we can measure a spectroscopic redshift :-) More on this later...

The rows and columns are showing the pixel scale, but there is information in the header of the FITS file that tells us how to convert from pixels to wavelength so we can precisely analyze the data. Let's do this with the 1D spectrum...

In [ ]:
# Lets compare the header of the 2D spectrum with the header of the 1D spectrum

#Code for 1D header here

hdu1[0].header

That's a lot of info! Take a gander and see what you can find. (Remind us to talk about optimal vs boxcar extraction sometime if we haven't already) 

Of interest to us at the moment is our wavelength scaling info. We will use the information in the 1D spectra header to make our wavelength array.

Of particular interest to us are the following keywords in the header:

CRVAL1: This tells us the starting wavelength for our array \
CDELT1: This tells us the spacing between wavelengths (ie: the $\Delta \lambda$ between different pixels)\
NAXIS1: The length of the wavelength array

In [ ]:
spec_header = hdu1[1].header

In [ ]:
# Translate the header data into a wavelength array
# Generate a wavelength array using the optimal extraction extension
# The starting value fo the wavelength array is located at
# CRVAL1 and goes the length of the array using NAXIS1 in steps of CDELT1
#       ENTRY1
# Wavelength = CRVAL1 + CDELT1, ENTRY1 + CDELT1, ENTRY2 + CDELT1, ...,  

start_wave = spec_header['CRVAL1']
length_of_arr = spec_header['NAXIS1']
delta_wave = spec_header['CDELT1']

wavelength = start_wave + np.arange(0, length_of_arr, 1) * delta_wave

#Use your knowledge of Astropy hdu objects to get the spectral data
spec1d = hdu1[1].data

Great! Now we have the data for the x-axis (wavelength) and for the y-axis (spectrum). Let's plot!

In [ ]:
# plot the 1D spectrum

plt.plot(wavelength, spec1d)
plt.xlabel(r'Wavelength $[\AA]$')
plt.ylabel(r'Signal [$erg\, s^{-1}\, cm^{-2}\, \AA^{-1}$]')

Uh...what does it mean? Basically, the way `matplotlib` has scaled this data is terrible. But that's fair, because spectra have huge noise spikes that we don't care about. So, we gotta scale better!

In [ ]:
# plot the 1D spectrum, now with a better scale
### note, another way to do this is to multiply the signal by a factor
### here I would use 1e17 and then set ylim from -1 to 1

plt.plot(wavelength, spec1d)
plt.xlabel(r'Wavelength $[\AA]$')
plt.ylabel(r'Signal [$erg\, s^{-1}\, cm^{-2}\, \AA^{-1}$]')

#Play around with a scaling below so that you can see the emission features of the spectrum 
#very prominently 

plt.xlim(15000,18000)
plt.ylim(-1e-17,1e-17)

There's our signal!! And you may start to notice some features...it'll get easier with practice looking at spectroscopic data. And using the 1D and 2D together is essential to pick out the features in 1D!

# Fitting for a redshift

Our next task is to figure out what lines we're seeing. Load in the provided line list and let's see if we can figure it out.

The following cell loads in common emission, absoprtions and sky lines at various rest-frame wavelengths data from the Sloan digital Sky Survey (SDSS).

In [ ]:
# list of spectral lines (originally from SDSS)
### shows the vacuum (rest) wavelength, the species, and the type of line

lines = pd.read_csv('linelist.csv',delimiter=",",comment='#') # all lines
em = lines[lines['type']=='Emission'] # emission lines
ab = lines[lines['type']=='Absorption'] # absorption lines
sky = lines[lines['type']=='Sky'] # absorption lines

em

We will use these lines to figure out the redshift of the galaxy we are looking at. Redshift is an astronomical term that basically tells us how much the light from a galaxy has been stretched due to the expansion of the universe. I more nearby galaxy will have little stretch and will have a low redshift where a galaxy further away will have its light stretched more and will have a higher redshift. 

Our task is to take the lines that we loaded in and determine the redshift of the source. We will do this by guessing where the lines should be for a given redshift.

Your task is to input a guess for redshift in for zguess below and try to match up line features in the spectrum, Matching the emission features with spikes, and absorption features with dips.

In [ ]:
# pay attention to the scaling here!
# your task: change zguess until the lines match up with the features :-) 

zguess = 2.3074 ## adjust the redshift!

plt.plot(wavelength, spec1d*1e17)
plt.xlabel(r'Wavelength $[\AA]$')
plt.ylabel(r'Signal [$10^{-17} erg\, s^{-1}\, cm^{-2}\, \AA^{-1}$]')

plt.xlim(15500,17000)
plt.ylim(-0.5,1.5)

#Plotting the lines
plt.plot(ab['lambda']*(1+zguess),0*np.ones_like(ab['lambda']),'kv',ms=3,label='absorption')
plt.plot(em['lambda']*(1+zguess),0*np.ones_like(em['lambda']),'k^',ms=3,label='emission')
plt.plot(sky['lambda']*(1+zguess),0*np.ones_like(sky['lambda']),'kd',ms=3,label='sky')

#this is code that draws an arrow for us to visually see them lining up
for i in range(len(lines['lambda'])):
    plt.annotate(str(lines['species'][i]+', '+str(lines['lambda'][i])),xy=(lines['lambda'][i]*(1+zguess), 0),\
                xytext=(lines['lambda'][i]*(1+zguess)-10, 1),arrowprops=dict(arrowstyle="-",),size=7,rotation=90)
plt.legend()

# More Exact Redshift Estimate

Doing things by eye is a great first start but our eyes cannot see down to very fine details. For exact science and a more robust prediction on the redshift we need to use fitting techniques to fit the emission line and get a better measurment of the peak emission wavelength. For that we will use $\textbf{Model Fitting}$.

Your goal is to fit an emission line of your choosing with a Gaussian and provide us with the wavelength and the spectrosopic redshift of the source.

In [ ]:
# First step is to make the model of our Gaussian 
# Complete the function definition of this Gaussian
# Look back to Day 3 for definition of a Gaussian or feel free to google it
# Just note what each parameter in this gaussian is in the context of the spectra

# x = wavelength
# sigma = broadness of the emission line
# A = amplitude of the emission line
# mu = centered mean of the gaussian

def gaussian(x, A, mu, sigma):
    '''
    Function declaration for a Gaussian
    
    Input(s)
    ----------
    
    
    Output(s)
    -------------
    
    '''
    
    exponent = (x - mu)**2/(2*sigma**2)
    
    return A * np.exp(-exponent)

In [ ]:
# fit a gaussian to find line center
# make a cut so the fit only uses data around the line we want to fit

window = 100

index = np.where((wavelength > (16400 - window)) & (wavelength < (16400 + window)))

wav = wavelength[index]
spec = spec1d[index] * 1e17

In [ ]:
#from looking at the plot looks like amplitude is .25, center is at 16400, 
#looks to be 2 angstroms wide
guess = .25, 16400, 2
bounds = ((0, 16400 - 25, 0), (np.inf, 16400 + 25, np.inf))
popt, pcov = curve_fit(gaussian, wav, spec, p0=guess, bounds=bounds)

obswav = popt[1]
obswav_error = np.sqrt(pcov[1, 1])
print(f'Observed wavelength of OII [5008] is {obswav:.3f} +/- {obswav_error:.3f}')

In [ ]:
# measure the redshift

restwav = 
redshift = obswav / restwav - 1
print(f'The redshift is {redshift:.4f} ')